In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.models import Model

In [38]:
noise_size = 30

X = x_train.astype(float) / 255
X_ = x_test.astype(float) / 255

In [39]:
in_layer = Input(shape=(noise_size,))
dense = Dense(units=100, activation="tanh")(in_layer)
dense = Dense(units=400, activation="tanh")(dense)
dense = Dense(units=784, activation="sigmoid")(dense)
out_layer = Reshape(target_shape=(28, 28))(dense)

In [41]:
gen = Model(in_layer, out_layer)

In [45]:
gen.compile(optimizer="adam", loss="binary_crossentropy")

In [46]:
gen.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 30)]              0         
_________________________________________________________________
dense_39 (Dense)             (None, 100)               3100      
_________________________________________________________________
dense_40 (Dense)             (None, 400)               40400     
_________________________________________________________________
dense_41 (Dense)             (None, 784)               314384    
_________________________________________________________________
reshape_11 (Reshape)         (None, 28, 28)            0         
Total params: 357,884
Trainable params: 357,884
Non-trainable params: 0
_________________________________________________________________


In [51]:
in_layer = Input(shape=(28, 28,))
reshape = Reshape(target_shape=(784,))(in_layer)
dense = Dense(units=400, activation="tanh")(reshape)
dense = Dense(units=100, activation="tanh")(dense)
dense = Dense(units=10, activation="tanh")(dense)
out_layer = Dense(units=1, activation="sigmoid")(dense)

In [52]:
disc = Model(in_layer, out_layer)

In [55]:
disc.compile(optimizer="adam", loss="binary_crossentropy")

In [56]:
disc.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 28, 28)]          0         
_________________________________________________________________
reshape_13 (Reshape)         (None, 784)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 400)               314000    
_________________________________________________________________
dense_47 (Dense)             (None, 100)               40100     
_________________________________________________________________
dense_48 (Dense)             (None, 10)                1010      
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 11        
Total params: 355,121
Trainable params: 355,121
Non-trainable params: 0
____________________________________________________

In [57]:
in_layer = Input(shape=(noise_size,))
gen_layer = gen(in_layer)
disc_layer = disc(gen_layer)

In [58]:
gan = Model(in_layer, disc_layer)

In [59]:
gan.compile(optimizer="adam", loss="binary_crossentropy")

In [60]:
gan.summary()

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 30)]              0         
_________________________________________________________________
model_12 (Model)             (None, 28, 28)            357884    
_________________________________________________________________
model_14 (Model)             (None, 1)                 355121    
Total params: 713,005
Trainable params: 713,005
Non-trainable params: 0
_________________________________________________________________


In [73]:
def display(images):
    plt.figure(figsize=(5, 5))
    for i in range(1, 26):
        plt.subplot(5, 5, i)
        plt.imshow(images[i], cmap="gray")
    plt.show()
   

In [61]:
for i in range(100):
    noise = np.random.randn(1000, noise_size)
    fake_images = gen.predict(noise)

    real_images = X[np.random.randint(0, len(X), 1000)]

    disc.trainable = True

    disc.fit(fake_images, np.zeros([1000]), verbose=0)
    disc.fit(real_images, np.ones([1000]), verbose=0)

    disc.trainable = False

    gan.fit(noise, np.ones([1000]) * .9, verbose=0)
    
    if i % 10 == 0:
        display(fake_images)



In [74]:
noise = np.random.randn(1000, noise_size)
fake_images = gen.predict(noise)

In [ ]:
display(fake_images)